In [1]:
import cv2 as cv
import numpy as np
import shutil
from sklearn.cluster import KMeans
import random
import os

indstillinger

In [2]:
full_set_plader = "Data/KD hele plader"
train_set_plader = "Data/KD train plader"
test_set_plader = "Data/KD Test plader"
train_set_tiles = "Data/KD train tiles" 

train_set_tiles_mixed = train_set_tiles + "/blandet"

test_size=0.2

crop_percentage = 0
circle_size = 0.0

n_clusters=7

Laver test split af hele plader

In [3]:
def copy_files(files, destination):
    for f in files:
        shutil.copy(os.path.join(full_set_plader, f), os.path.join(destination, f))

In [4]:
# Sikre, at destinationsmapperne eksisterer
os.makedirs(train_set_plader, exist_ok=True)
os.makedirs(train_set_plader, exist_ok=True)

# Hent alle filnavne fra data-mappen
all_files = [f for f in os.listdir(full_set_plader) if os.path.isfile(os.path.join(full_set_plader, f))]

# Bland listen i state 42 og lav et tilfældigt split
random.Random(42).shuffle(all_files)
split_point = int((1 - test_size) * len(all_files))

# Fordel filerne i train og test mapper
train_files = all_files[:split_point]
test_files = all_files[split_point:]

# Kopier filerne
copy_files(train_files, train_set_plader)
copy_files(test_files, test_set_plader)

Laver tile split

In [5]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

In [6]:
all_tiles = []

for filename in os.listdir(train_set_plader):
    image_path = os.path.join(train_set_plader, filename)

    if not os.path.isfile(image_path):
        print(f"Image not found: {image_path}")
        continue
    
    image = cv.imread(image_path)
    tiles = get_tiles(image)
    
    for y, row in enumerate(tiles):
        for x, tile in enumerate(row):
            #tile = zoom_tile(tile)
            #tile = remove_circle_from_tile(tile)
            all_tiles.append(tile)

In [7]:
hsv_colors = []
for tile in all_tiles:
    hsv = cv.cvtColor(tile, cv.COLOR_BGR2HSV)
    avg_color_per_row = np.average(hsv, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    hsv_colors.append(avg_color)

# Anvend K-means clustering på de fundne farver
kmeans = KMeans(n_clusters, random_state=0).fit(hsv_colors)

# Opret mapper til de forskellige clusters, hvis de ikke allerede findes
for i in range(n_clusters):
    os.makedirs(f'{train_set_tiles}/cluster_{i}', exist_ok=True)

# Gem hvert billede i den tilsvarende mappe baseret på dets cluster
for idx, tile in enumerate(all_tiles):
    cluster_index = kmeans.labels_[idx]
    tile_path = f'{train_set_tiles}/cluster_{cluster_index}/{idx}.jpg'
    cv.imwrite(tile_path, tile)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
